In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql 
with USA_tracks_sold as
(SELECT 
 il.*
 FROM invoice_line il
 inner join invoice i on i.invoice_id = il.invoice_id
 inner join customer c on  i.customer_id = c.customer_id
 where c.country = "USA"
)
select
g.name genre,
CAST(count(uts.invoice_line_id) as float)/(select count(*) from USA_tracks_sold) as percentage_sold
from USA_tracks_sold uts 
inner join track t on t.track_id = uts.track_id
inner join genre g on g.genre_id = t.genre_id
group by 1
order by 2 desc
limit 10



Done.


genre,percentage_sold
Rock,0.5337773549000951
Alternative & Punk,0.12369172216936251
Metal,0.11798287345385347
R&B/Soul,0.05042816365366318
Blues,0.03425309229305423
Alternative,0.03330161750713606
Latin,0.02093244529019981
Pop,0.02093244529019981
Hip Hop/Rap,0.019029495718363463
Jazz,0.013320647002854425


When looking at total purchases per genre we conclude that the top three genres are Rock, Alternative & Punk, and Metal. Based on the results, the three albums that should be purchased are:
- Red Tone
- Slim Jim Bites
- Meteor and the Girls

# Analyzing Employee Sales Performance

In [5]:
%%sql
with customer_total_purchases as
(select
i.customer_id,
 c.support_rep_id,
 sum(i.total) total_purchase
from invoice i
inner join customer c on c.customer_id = i.customer_id
 group by 1,2
)
select 
e.first_name || " " || e.last_name as employee_name ,
sum(ctp.total_purchase) total_purchases
from customer_total_purchases ctp
inner join employee e on e.employee_id = ctp.support_rep_id
group by 1


Done.


employee_name,total_purchases
Jane Peacock,1731.5099999999998
Margaret Park,1584.0000000000002
Steve Johnson,1393.92


The sales agent with the highest sales amount assigned was Jane Peacock. Following her Margaret Park sold the second most out of the employees.

# Analyzing Sales by Country

In [6]:
%%sql
with country_sales as 
(select
    c.country country,
    count(distinct(c.customer_id)) total_customers,
    round(sum(i.total)) total_sales,
    count(i.invoice_id) as total_orders
from customer c
inner join invoice i on c.customer_id = i.customer_id
group by 1),

country_sales_other as
(select
    case
        when total_customers = 1 then "other"
        else country
        end as country_identification,
    total_customers,
    total_sales,
    total_orders
from
    country_sales),

country_sales_agg as 
(select
    country_identification,
    sum(total_customers) total_customers,
    sum(total_sales) total_sales,
    sum(total_orders) total_orders
from country_sales other
group by 1),

country_variables as
(select
    country_identification country,
    total_customers,
    total_sales,
    round(cast(total_sales / total_customers as float),2) average_sales_per_customer,
    round(cast(total_sales / total_orders as float),2) average_order_value
from country_sales_agg)

select
    country, 
    total_customers,
    total_sales,
    average_sales_per_customer,
    average_order_value
from country_variables



     
    

(sqlite3.OperationalError) no such column: country_identification
[SQL: with country_sales as 
(select
    c.country country,
    count(distinct(c.customer_id)) total_customers,
    round(sum(i.total)) total_sales,
    count(i.invoice_id) as total_orders
from customer c
inner join invoice i on c.customer_id = i.customer_id
group by 1),

country_sales_other as
(select
    case
        when total_customers = 1 then "other"
        else country
        end as country_identification,
    total_customers,
    total_sales,
    total_orders
from
    country_sales),

country_sales_agg as 
(select
    country_identification,
    sum(total_customers) total_customers,
    sum(total_sales) total_sales,
    sum(total_orders) total_orders
from country_sales other
group by 1),

country_variables as
(select
    country_identification country,
    total_customers,
    total_sales,
    round(cast(total_sales / total_customers as float),2) average_sales_per_customer,
    round(cast(total_sales / total_or

# Albums vs Individual Tracks

In [22]:
%%sql
WITH number_of_tracks_per_invoice AS (
  SELECT 
    il.invoice_id,
    il.track_id,
    COUNT(quantity) number_of_tracks
  FROM invoice_line il
  GROUP BY 1, 2
),
tracks_per_album AS (
  SELECT
    t.album_id,
    t.track_id,
    a.title
  FROM track t
  INNER JOIN album a ON a.album_id = t.album_id
  GROUP BY 1, 2, 3
)
SELECT 
  a.title,
  tpa.track_id,
  ntpi.number_of_tracks
FROM tracks_per_album tpa
INNER JOIN number_of_tracks_per_invoice ntpi ON tpa.track_id = ntpi.track_id;




(sqlite3.OperationalError) no such column: a.title
[SQL: WITH number_of_tracks_per_invoice AS (
  SELECT 
    il.invoice_id,
    il.track_id,
    COUNT(quantity) number_of_tracks
  FROM invoice_line il
  GROUP BY 1, 2
),
tracks_per_album AS (
  SELECT
    t.album_id,
    t.track_id,
    a.title
  FROM track t
  INNER JOIN album a ON a.album_id = t.album_id
  GROUP BY 1, 2, 3
)
SELECT 
  a.title,
  tpa.track_id,
  ntpi.number_of_tracks
FROM tracks_per_album tpa
INNER JOIN number_of_tracks_per_invoice ntpi ON tpa.track_id = ntpi.track_id;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
